# ETL Example

In this example, we'll write a full ETL pipeline for the GDP data. That means we'll extract the World Bank data (taken from https://data.worldbank.org), transform the data, and load the data all in one go. In other words, we'll want one Python script that can do the entire process.

Why would we want to do this? Imagine working for a company that creates new data every day. As new data comes in, we'll want to write software that periodically and automatically extracts, transforms, and loads the data.

We'll extract the GDP data one line at a time. We'll then transform that line of data and load the results into a SQLite database.

Here is an explanation of how this Jupyter notebook is organized:
- The first cell connects to a SQLite database called worldbank.db and creates a table to hold the gdp data.
- The second cell has a function called extract_line(). This function is a [Python generator](https://wiki.python.org/moin/Generators). A generator is like a regular function except instead of a return statement, a generator has a yield statement. Generators allow us to use functions in a for loop. In essence, this function will allow us to read in a data file one line at a time, run a transformation on that row of data, and then move on to the next row in the file.
3. The third cell contains a function called transform_indicator_data(). This function receives a line from the csv file and transforms the data in preparation for a load step.
4. The fourth cell contains a function called load_indicator_data(), which loads the trasnformed data into the gdp table in the worldbank.db database.
5. The fifth cell runs the ETL pipeilne
6. The sixth cell runs a query against the database to make sure everything worked correctly.

In [1]:
# run this cell to create a database and a table, called gdp, to hold the gdp data

import sqlite3

# connect to the database
# the database file will be worldbank.db
# note that sqlite3 will create this database file if it does not exist already
conn = sqlite3.connect('worldbank.db')

# get a cursor
cur = conn.cursor()

# drop the test table in case it already exists
cur.execute("DROP TABLE IF EXISTS gdp")

# create the test table including project_id as a primary key
cur.execute("CREATE TABLE gdp (countryname TEXT, countrycode TEXT, year INTEGER, gdp REAL, PRIMARY KEY (countrycode, year));")

conn.commit()
conn.close()

In [2]:
# Generator for reading in one line at a time
# generators are useful for data sets that are too large to fit in RAM

def extract_lines(file):
    while True:
        line = file.readline()
        if not line:
            break
        yield line

In [3]:
import pandas as pd
import numpy as np
import sqlite3

# transform the indicator data
def transform_indicator_data(data, colnames):
    
    # get rid of quote marks
    for i, datum in enumerate(data):
        data[i] = datum.replace('"','')
    
    country = data[0]
    
    # filter out values that are not countries
    non_countries = ['World',
     'High income',
     'OECD members',
     'Post-demographic dividend',
     'IDA & IBRD total',
     'Low & middle income',
     'Middle income',
     'IBRD only',
     'East Asia & Pacific',
     'Europe & Central Asia',
     'North America',
     'Upper middle income',
     'Late-demographic dividend',
     'European Union',
     'East Asia & Pacific (excluding high income)',
     'East Asia & Pacific (IDA & IBRD countries)',
     'Euro area',
     'Early-demographic dividend',
     'Lower middle income',
     'Latin America & Caribbean',
     'Latin America & the Caribbean (IDA & IBRD countries)',
     'Latin America & Caribbean (excluding high income)',
     'Europe & Central Asia (IDA & IBRD countries)',
     'Middle East & North Africa',
     'Europe & Central Asia (excluding high income)',
     'South Asia (IDA & IBRD)',
     'South Asia',
     'Arab World',
     'IDA total',
     'Sub-Saharan Africa',
     'Sub-Saharan Africa (IDA & IBRD countries)',
     'Sub-Saharan Africa (excluding high income)',
     'Middle East & North Africa (excluding high income)',
     'Middle East & North Africa (IDA & IBRD countries)',
     'Central Europe and the Baltics',
     'Pre-demographic dividend',
     'IDA only',
     'Least developed countries: UN classification',
     'IDA blend',
     'Fragile and conflict affected situations',
     'Heavily indebted poor countries (HIPC)',
     'Low income',
     'Small states',
     'Other small states',
     'Not classified',
     'Caribbean small states',
     'Pacific island small states']
    
    if country not in non_countries:
        data_array = np.array(data, ndmin=2)
        data_array.reshape(1, 63)
        df = pd.DataFrame(data_array, columns=colnames).replace('', np.nan)
        df.drop(['\n', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)

        # Reshape the data sets so that they are in long format
        df_melt = df.melt(id_vars=['Country Name', 'Country Code'], 
                            var_name='year', 
                            value_name='gdp')
        
        results = []
        for index, row in df_melt.iterrows():
            country, countrycode, year, gdp = row
            if str(gdp) != 'nan':
                results.append([country, countrycode, year, gdp])
        return results
    


In [4]:
def load_indicator_data(results):
    conn = sqlite3.connect('worldbank.db')
    cur = conn.cursor()
    if results:
        for result in results:
            countryname, countrycode, year, gdp = result

            sql_string = 'INSERT INTO gdp (countryname, countrycode, year, gdp) VALUES ("{}", "{}", {}, {});'.format(countryname, countrycode, year, gdp)

            # connect to database and execute query
            try:
                cur.execute(sql_string)
            except Exception as e:
                print('error occurred:', e, result)
            
    conn.commit()
    conn.close()
    
    return None

In [5]:
# Execute this code cell to run the ETL pipeline

with open('../data/gdp_data.csv') as f:
    for line in extract_lines(f):
        data = line.split(',')
        if len(data) == 63:
            if data[0] == '"Country Name"':
                colnames = []
                # get rid of quote marks
                for i, datum in enumerate(data):
                    colnames.append(datum.replace('"',''))
            else:
                # transform and load the line of indicator data
                results = transform_indicator_data(data, colnames)
                load_indicator_data(results)

In [6]:
# Execute this code cell to output the values in the gdp table

# connect to the database
# the database file will be worldbank.db
# note that sqlite3 will create this database file if it does not exist already
conn = sqlite3.connect('worldbank.db')

# get a cursor
cur = conn.cursor()

# create the test table including project_id as a primary key
df = pd.read_sql("SELECT * FROM gdp", con=conn)

conn.commit()
conn.close()

df

countryname countrycode  year           gdp
0           Aruba         ABW  1994  1.330168e+09
1           Aruba         ABW  1995  1.320670e+09
2           Aruba         ABW  1996  1.379888e+09
3           Aruba         ABW  1997  1.531844e+09
4           Aruba         ABW  1998  1.665363e+09
5           Aruba         ABW  1999  1.722799e+09
6           Aruba         ABW  2000  1.873453e+09
7           Aruba         ABW  2001  1.920263e+09
8           Aruba         ABW  2002  1.941095e+09
9           Aruba         ABW  2003  2.021302e+09
10          Aruba         ABW  2004  2.228279e+09
11          Aruba         ABW  2005  2.331006e+09
12          Aruba         ABW  2006  2.421475e+09
13          Aruba         ABW  2007  2.623726e+09
14          Aruba         ABW  2008  2.791961e+09
15          Aruba         ABW  2009  2.498933e+09
16          Aruba         ABW  2010  2.467704e+09
17          Aruba         ABW  2011  2.584464e+09
18    Afghanistan         AFG  1960  5.377778e+08
19    Afghanistan         AFG  1961  5.488889e+08
20    Afghanistan         AFG  1962  5.466667e+08
21    Afghanistan         AFG  1963  7.511112e+08
22    Afghanistan         AFG  1964  8.000000e+08
23    Afghanistan         AFG  1965  1.006667e+09
24    Afghanistan         AFG  1966  1.400000e+09
25    Afghanistan         AFG  1967  1.673333e+09
26    Afghanistan         AFG  1968  1.373333e+09
27    Afghanistan         AFG  1969  1.408889e+09
28    Afghanistan         AFG  1970  1.748887e+09
29    Afghanistan         AFG  1971  1.831109e+09
...           ...         ...   ...           ...
8684     Zimbabwe         ZWE  1988  7.814784e+09
8685     Zimbabwe         ZWE  1989  8.286323e+09
8686     Zimbabwe         ZWE  1990  8.783817e+09
8687     Zimbabwe         ZWE  1991  8.641482e+09
8688     Zimbabwe         ZWE  1992  6.751472e+09
8689     Zimbabwe         ZWE  1993  6.563813e+09
8690     Zimbabwe         ZWE  1994  6.890675e+09
8691     Zimbabwe         ZWE  1995  7.111271e+09
8692     Zimbabwe         ZWE  1996  8.553147e+09
8693     Zimbabwe         ZWE  1997  8.529572e+09
8694     Zimbabwe         ZWE  1998  6.401968e+09
8695     Zimbabwe         ZWE  1999  6.858013e+09
8696     Zimbabwe         ZWE  2000  6.689958e+09
8697     Zimbabwe         ZWE  2001  6.777385e+09
8698     Zimbabwe         ZWE  2002  6.342116e+09
8699     Zimbabwe         ZWE  2003  5.727592e+09
8700     Zimbabwe         ZWE  2004  5.805598e+09
8701     Zimbabwe         ZWE  2005  5.755215e+09
8702     Zimbabwe         ZWE  2006  5.443896e+09
8703     Zimbabwe         ZWE  2007  5.291950e+09
8704     Zimbabwe         ZWE  2008  4.415703e+09
8705     Zimbabwe         ZWE  2009  8.621574e+09
8706     Zimbabwe         ZWE  2010  1.014186e+10
8707     Zimbabwe         ZWE  2011  1.209845e+10
8708     Zimbabwe         ZWE  2012  1.424249e+10
8709     Zimbabwe         ZWE  2013  1.545177e+10
8710     Zimbabwe         ZWE  2014  1.589105e+10
8711     Zimbabwe         ZWE  2015  1.630467e+10
8712     Zimbabwe         ZWE  2016  1.661996e+10
8713     Zimbabwe         ZWE  2017  1.784582e+10

[8714 rows x 4 columns]